In [1]:
from typing import Callable, Any, Set, Tuple, List, FrozenSet, Dict
from functools import lru_cache
from sentence_transformers import SentenceTransformer, util
import numpy as np
import statistics

# --- SBERT Helpers ---
sbert_model = SentenceTransformer("all-MiniLM-L6-v2")
_seen_debugs = {
    level: set() for level in [
        "literal_flat", "literal_weighted",
        "clause_flat", "clause_weighted",
        "formula"]
}
_debug_summary = {
    level: [] for level in [
        "literal_flat", "literal_weighted",
        "clause_flat", "clause_weighted",
        "formula"]
}

@lru_cache(maxsize=2048)
def get_embedding(text: str):
    return sbert_model.encode(text, convert_to_tensor=True)

def normalize_str(s: str) -> str:
    return s.lower().replace("_", " ").replace("(", "").replace(")", "").replace("not ", "¬").strip()

def sim_sbert_normalized(a: str, b: str) -> float:
    a_norm, b_norm = normalize_str(a), normalize_str(b)
    return util.cos_sim(get_embedding(a_norm), get_embedding(b_norm)).item()

def agg_max(scores: List[float]) -> float:
    return max(scores) if scores else 0.0

def agg_avg(scores: List[float]) -> float:
    return sum(scores) / len(scores) if scores else 0.0

def agg_softmax(scores: List[float], temperature: float = 2.5) -> float:
    if not scores:
        return 0.0
    exp_scores = [np.exp(s * temperature) for s in scores]
    softmax_weights = [e / sum(exp_scores) for e in exp_scores]
    return sum(w * s for w, s in zip(softmax_weights, scores))

def pointwise_similarity_weighted(
    args1: List[str],
    args2: List[str],
    simC: Callable[[str, str], float],
    weights1: Dict[str, float],
    weights2: Dict[str, float],
    positional_weight: float = 0.8,
    unordered_weight: float = 0.2,
    diff: float = 0
) -> float:
    if not args1 or not args2:
        return 0.0

    min_len = min(len(args1), len(args2))
    sim_pos, total_weight_pos = 0.0, 0.0
    for i in range(min_len):
        a1, a2 = args1[i], args2[i]
        sim = simC(a1, a2)
        w1 = weights1.get(a1, 0.0)
        w2 = weights2.get(a2, 0.0)
        importance = w1 * w2
        sim_pos += importance * sim
        total_weight_pos += importance
    sim_pos = sim_pos / (total_weight_pos + diff) if total_weight_pos > 0 else 0.0

    sim_unordered, total_weight_unordered = 0.0, 0.0
    for a1 in args1:
        w1 = weights1.get(a1, 0.0)
        best_sim, best_imp = 0.0, 0.0
        for a2 in args2:
            w2 = weights2.get(a2, 0.0)
            sim = simC(a1, a2)
            importance = w1 * w2
            if sim > best_sim:
                best_sim, best_imp = sim, importance
        sim_unordered += best_sim * best_imp
        total_weight_unordered += best_imp

    sim_unordered = sim_unordered / (total_weight_unordered + diff) if total_weight_unordered > 0 else 0.0

    return positional_weight * sim_pos + unordered_weight * sim_unordered

def safe_sort_clause(c: FrozenSet[Tuple[str, Tuple[str, ...]]]) -> List:
    return sorted(list(c), key=lambda lit: (lit[0], lit[1]))

def record_debug(level: str, key: frozenset, entry: Tuple):
    if key not in _seen_debugs[level]:
        _seen_debugs[level].add(key)
        _debug_summary[level].append(entry)

def simLiteral_flat(
    obj1: Tuple[str, Tuple[str, ...]],
    obj2: Tuple[str, Tuple[str, ...]],
    debug: dict = None,
    diff: float = 0.0
) -> float:
    pred1, args1 = obj1
    pred2, args2 = obj2

    sim_pred = sim_sbert_normalized(pred1, pred2)
    sim_args = pointwise_similarity_weighted(
        list(args1), list(args2), sim_sbert_normalized,
        weights1={k: 1.0 for k in args1},
        weights2={k: 1.0 for k in args2},
        diff=diff
    )

    score = 0.5 * sim_pred + 0.5 * sim_args

    if debug and debug.get("literal", False):
        key = frozenset({obj1, obj2})
        canonical = tuple(sorted([obj1, obj2]))
        record_debug("literal_flat", key, (*canonical, sim_pred, sim_args, score))

    return score

def simLiteral(
    obj1: Tuple[str, Tuple[str, ...]],
    obj2: Tuple[str, Tuple[str, ...]],
    predicate_weights: Dict[str, float],
    constant_weights: Dict[str, float],
    debug: dict = None,
    diff: float = 0.0
) -> float:
    pred1, args1 = obj1
    pred2, args2 = obj2

    sim_pred = sim_sbert_normalized(pred1, pred2)
    imp_pred = predicate_weights.get(pred1, 0.0) * predicate_weights.get(pred2, 0.0)
    num = sim_pred * imp_pred
    denom = imp_pred

    sim_args = pointwise_similarity_weighted(
        list(args1), list(args2), sim_sbert_normalized,
        weights1=constant_weights,
        weights2=constant_weights,
        diff=diff
    )

    imp_args_total = sum(
        constant_weights.get(a1, 0.0) * constant_weights.get(a2, 0.0)
        for a1 in args1 for a2 in args2
    )
    num += sim_args * imp_args_total
    denom += imp_args_total

    score = num / (denom + diff) if denom > 0 else 0.0

    if debug and debug.get("literal", False):
        key = frozenset({obj1, obj2})
        canonical = tuple(sorted([obj1, obj2]))
        record_debug("literal_weighted", key, (*canonical, sim_pred, sim_args, score))

    return score

def generalized_tversky_similarity(
    X: Set[Any],
    Y: Set[Any],
    sim: Callable[[Any, Any], float],
    alpha: float = 1.0,
    beta: float = 1.0,
    p: float = 1.0,
    agg: Callable[[List[float]], float] = max,
    debug: dict = None,
    debug_level: str = ""
) -> float:
    if not X or not Y:
        return 0.0

    def membership(x, Y):
        return agg([sim(x, y) ** p for y in Y])

    match_X = [membership(x, Y) for x in X]
    match_Y = [membership(y, X) for y in Y]

    a = (sum(match_X) + sum(match_Y)) / 2
    b = sum(1 - s for s in match_X)
    c = sum(1 - s for s in match_Y)

    denom = a + alpha * b + beta * c
    score = a / denom if denom > 0 else 0.0

    if debug and debug.get(debug_level, False):
        _debug_summary[debug_level].append(("TOTAL", a, b, c, score))
        seen = set()
        for x in X:
            for y in Y:
                key = frozenset([x, y])
                if key not in seen:
                    seen.add(key)
                    s = sim(x, y)
                    _debug_summary[debug_level].append(("clause_pair", x, y, s))  # ✓ Ajouter la similarité ici

    return score


def simFormulaSet(
    F1: Set[FrozenSet[Tuple[str, Tuple[str, ...]]]],
    F2: Set[FrozenSet[Tuple[str, Tuple[str, ...]]]],
    clause_weights1: Dict[FrozenSet[Tuple[str, Tuple[str, ...]]], float],
    clause_weights2: Dict[FrozenSet[Tuple[str, Tuple[str, ...]]], float],
    predicate_weights1: Dict[str, float],
    predicate_weights2: Dict[str, float],
    constant_weights1: Dict[str, float],
    constant_weights2: Dict[str, float],
    params: dict = None,
    debug: dict = None
) -> Tuple[float, Dict[str, List[Any]]]:
    for level in _seen_debugs:
        _seen_debugs[level].clear()
    for level in _debug_summary:
        _debug_summary[level].clear()

    predicate_weights = {**predicate_weights1, **predicate_weights2}
    constant_weights = {**constant_weights1, **constant_weights2}

    sum_preds = sum(predicate_weights1.values()) + sum(predicate_weights2.values())
    sum_consts = sum(constant_weights1.values()) + sum(constant_weights2.values())
    diff = 2 - sum_preds - sum_consts
    epsilon = 1e-4
    diff = 0.0 if abs(diff) < epsilon else diff

    all_clauses1 = sorted(F1, key=safe_sort_clause)
    all_clauses2 = sorted(F2, key=safe_sort_clause)

    # --- Flat clause similarities ---
    flat_similarities = {}
    for c1 in all_clauses1:
        for c2 in all_clauses2:
            sim = generalized_tversky_similarity(
                c1, c2,
                sim=simLiteral_flat,
                alpha=1.0, beta=1.0, p=1.0, agg=max,
                debug=debug,
                debug_level="clause_flat"
            )
            _debug_summary["clause_flat"].append(("clause_pair_score", c1, c2, sim))
            flat_similarities[(c1, c2)] = sim

    # --- Best clause matches ---
    best_matches = {}
    for c1 in all_clauses1:
        best_c2 = max(all_clauses2, key=lambda c2: flat_similarities[(c1, c2)])
        best_matches[(c1, best_c2)] = flat_similarities[(c1, best_c2)]
    for c2 in all_clauses2:
        best_c1 = max(all_clauses1, key=lambda c1: flat_similarities[(c1, c2)])
        best_matches[(best_c1, c2)] = flat_similarities[(best_c1, c2)]

    # --- Weighted clause similarities on best matches ---
    weighted_scores = []
    for (c1, c2), _ in best_matches.items():
        sim = generalized_tversky_similarity(
            c1, c2,
            sim=lambda l1, l2: simLiteral(
                l1, l2,
                predicate_weights=predicate_weights,
                constant_weights=constant_weights,
                debug=debug,
                diff=diff
            ),
            alpha=params.get("clause", {}).get("alpha", 0.5),
            beta=params.get("clause", {}).get("beta", 0.5),
            p=params.get("clause", {}).get("p", 1.0),
            agg=params.get("clause", {}).get("agg", max),
            debug=debug,
            debug_level="clause_weighted"
        )
        _debug_summary["clause_weighted"].append(("clause_pair_score", c1, c2, sim))
        w1 = clause_weights1.get(c1, 1.0)
        w2 = clause_weights2.get(c2, 1.0)
        weighted_scores.append((sim, (w1 + w2)/2))

    # --- Final weighted score ---
    total_weight = sum(w for _, w in weighted_scores)
    #total_weight = sum(1 for _, w in weighted_scores)
    #print(weighted_scores)
    final_score = sum(s * w for s, w in weighted_scores) / total_weight if total_weight > 0 else 0.0

    _debug_summary["formula"].append(("TOTAL", 0, 0, 0, final_score))
    _debug_summary["formula"].extend([
        ("formula_pair", c1, c2, flat_similarities[(c1, c2)])
        for (c1, c2) in best_matches.keys()
    ])

    return final_score, _debug_summary


In [2]:
def summarize_debug_output(summary: dict, debug: dict = None, top_n: int = 10):
    print("\n🔍 SUMMARY OF SIMILARITY COMPARISON\n")

    # 1. Literal FLAT
    if debug.get("literal", False):
        literals = summary.get("literal_flat", [])
        print("📌 LITERAL LEVEL (Flat)")
        print(f"  → Total literal pairs (flat): {len(literals)}")
        for (lit1, lit2, sim_pred, sim_args, score) in literals:
            print(f"    - {lit1} ⇄ {lit2}")
            print(f"      • predicate similarity: {sim_pred:.3f}")
            print(f"      • argument similarity : {sim_args:.3f}")
            print(f"      • literal similarity   : {score:.3f}")

    # 2. Literal WEIGHTED
    literals_w = summary.get("literal_weighted", [])
    if literals_w:
        print("\n📌 LITERAL LEVEL (Weighted)")
        print(f"  → Total literal pairs (weighted): {len(literals_w)}")
        for (lit1, lit2, sim_pred, sim_args, score) in literals_w:
            print(f"    - {lit1} ⇄ {lit2}")
            print(f"      • predicate similarity (weighted): {sim_pred:.3f}")
            print(f"      • argument similarity  (weighted): {sim_args:.3f}")
            print(f"      • literal similarity   (weighted): {score:.3f}")

    # 3. Clause FLAT
    clauses = summary.get("clause_flat", [])
    if clauses:
        print("\n📌 CLAUSE LEVEL (Flat)")
        print(f"  → Total clause pairs compared (flat): {len(clauses)}")
        for (_, clause1, clause2, score) in clauses:
            print(f"    - Clause 1: {clause1}")
            print(f"      Clause 2: {clause2}")
            print(f"      → Clause similarity: {score:.3f}")

    # 4. Clause WEIGHTED
    clauses_w = summary.get("clause_weighted", [])
    if clauses_w:
        print("\n📌 CLAUSE LEVEL (Weighted)")
        print(f"  → Total clause pairs (weighted best matches): {len(clauses_w)}")
        for (_, clause1, clause2, score) in clauses_w:
            print(f"    - Clause 1: {clause1}")
            print(f"      Clause 2: {clause2}")
            print(f"      → Weighted clause similarity: {score:.3f}")

    # 5. FORMULA
    formulas = summary.get("formula", [])
    print("\n📌 FORMULA LEVEL")
    totals = [e for e in formulas if isinstance(e, tuple) and e[0] == "TOTAL"]
    for (_, a, b, c, score) in totals:
        print("  → Final formula similarity score (weighted over best clause matches):")
        print(f"      Score = {score:.3f}")


In [3]:
# --- Données pondérées pour I_SA et I_SB ---
weighted_T1 = {
    "instances": frozenset({
        frozenset({("Teasing", ("Dog", "Monkey"))}),
        frozenset({("AtLocation", ("Dog", "Zoo"))}),
        frozenset({("AtLocation", ("Monkey", "Zoo"))}),
    }),
    "clause_weights": {
        frozenset({("Teasing", ("Dog", "Monkey"))}): 0.9,
        frozenset({("AtLocation", ("Dog", "Zoo"))}): 0.05,
        frozenset({("AtLocation", ("Monkey", "Zoo"))}): 0.05,
    },
    "predicate_weights": {
        "Teasing": 0.1,
        "AtLocation": 0.1,
    },
    "constant_weights": {
        "Dog": 0.35,
        "Monkey": 0.35,
        "Zoo": 0.1
    }
}

weighted_T2 = {
    "instances": frozenset({
        frozenset({("Teasing", ("Monkey", "Dog"))}),
        frozenset({("AtLocation", ("Dog", "Zoo"))}),
        frozenset({("AtLocation", ("Monkey", "Zoo"))}),
    }),
    "clause_weights": {
        frozenset({("Teasing", ("Monkey", "Dog"))}): 0.9,
        frozenset({("AtLocation", ("Dog", "Zoo"))}): 0.05,
        frozenset({("AtLocation", ("Monkey", "Zoo"))}): 0.05,
    },
    "predicate_weights": {
        "Teasing": 0.1,
        "AtLocation": 0.1,
    },
    "constant_weights": {
        "Dog": 0.35,
        "Monkey": 0.35,
        "Zoo": 0.1
    }
}

# --- Paramètres de similarité ---
params = {
    "clause": {
        "alpha": 0.5,
        "beta": 0.5,
        "p": 1,
        "agg": agg_softmax
    },
    "formula": {
        "alpha": 0.5,
        "beta": 0.5,
        "p": 1,
        "agg": agg_max
    }
}

# --- Flags de debug ---
debug_flags = {
    "literal": True,
    "clause": True,
    "formula": True
}

# --- Appel de la fonction de similarité complète ---
score, summary = simFormulaSet(
    F1=weighted_T1["instances"],
    F2=weighted_T2["instances"],
    clause_weights1=weighted_T1["clause_weights"],
    clause_weights2=weighted_T2["clause_weights"],
    predicate_weights1=weighted_T1["predicate_weights"],
    predicate_weights2=weighted_T2["predicate_weights"],
    constant_weights1=weighted_T1["constant_weights"],
    constant_weights2=weighted_T2["constant_weights"],
    params=params,
    debug=debug_flags
)

print(f"\n🔷 FINAL SIMILARITY SCORE: {score:.3f}")
# --- Affichage du résumé complet ---
summarize_debug_output(summary, debug=debug_flags)





🔷 FINAL SIMILARITY SCORE: 0.623

🔍 SUMMARY OF SIMILARITY COMPARISON

📌 LITERAL LEVEL (Flat)
  → Total literal pairs (flat): 0

📌 LITERAL LEVEL (Weighted)
  → Total literal pairs (weighted): 3
    - ('AtLocation', ('Dog', 'Zoo')) ⇄ ('AtLocation', ('Dog', 'Zoo'))
      • predicate similarity (weighted): 1.000
      • argument similarity  (weighted): 1.000
      • literal similarity   (weighted): 1.000
    - ('AtLocation', ('Monkey', 'Zoo')) ⇄ ('AtLocation', ('Monkey', 'Zoo'))
      • predicate similarity (weighted): 1.000
      • argument similarity  (weighted): 1.000
      • literal similarity   (weighted): 1.000
    - ('Teasing', ('Dog', 'Monkey')) ⇄ ('Teasing', ('Monkey', 'Dog'))
      • predicate similarity (weighted): 1.000
      • argument similarity  (weighted): 0.573
      • literal similarity   (weighted): 0.581

📌 CLAUSE LEVEL (Flat)
  → Total clause pairs compared (flat): 9
    - Clause 1: frozenset({('AtLocation', ('Dog', 'Zoo'))})
      Clause 2: frozenset({('AtLocation', (